In [ ]:
#!pip install salmon-lm==1.0.3

In [ ]:
!git clone https://github.com/ajboyd2/salmon.git

Cloning into 'salmon'...
remote: Enumerating objects: 965, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 965 (delta 71), reused 68 (delta 37), pack-reused 854
Receiving objects: 100% (965/965), 9.54 MiB | 31.71 MiB/s, done.
Resolving deltas: 100% (581/581), done.


In [ ]:
cd salmon

/content/salmon


In [ ]:
!git pull

Already up to date.


In [ ]:
#!git checkout remotes/origin/XY-implementation
!python setup.py build
!python setup.py install

Note: checking out 'remotes/origin/XY-implementation'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at f5d4fec Fixed inplace modification of input data.
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
running build
running build_py
creating build
creating build/lib
creating build/lib/salmon
copying salmon/building.py -> build/lib/salmon
copying salmon/transformation.py -> build/lib/salmon
copying salmon/test.py -> build/lib/sal

In [ ]:
!pip install rpy2 
%load_ext rpy2.ipython

In [ ]:
import pandas as pd
from salmon import *
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
from tabulate import tabulate
import numpy as np
import cProfile

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv("data/harris.csv")

In [ ]:
%%R
data_r <- read.csv("data/harris.csv")

In [ ]:
nums = np.random.randint(0,50000,size=(10000,1000))
data2 = pd.DataFrame(nums)
y = data2.loc[:,0]
X = data2.iloc[:,1:]
cols = data2.columns

In [ ]:
%R -i data2,cols colnames(data2) <- unlist(cols);
%R rdf <- data.frame(data2);
%R y = rdf$X0
%R X = data.matrix(rdf[,-1])

array([[12916, 24464, 24850, ...,  2558, 12810, 28685],
       [42335,  9706,  3754, ...,  3909, 16846, 17665],
       [31640,  3916, 46127, ...,  8527, 33160, 27781],
       ...,
       [ 4812, 22980,  6296, ..., 18749, 38691, 28466],
       [  639, 20021, 29387, ...,  6808, 16144,  4016],
       [26776, 16674,  9809, ..., 35624, 41080, 20652]], dtype=int32)

In [ ]:
## salmon fit data
start_time = time.time()
explanatory = Q("Bsal")
response = Q("Sal77")
salmon_model = LinearModel(explanatory, response)
salmon_model.fit(data)
end_time = time.time()
elapsed_time_fit_sal = (end_time - start_time) * 1000

## statsmodels fit data
start_time = time.time()
statsmodels_results = smf.ols("Sal77 ~ Bsal", data=data).fit()
statsmodels_results.summary()
end_time = time.time()
elapsed_time_fit_sm = (end_time - start_time) * 1000

## salmon predict
start_time = time.time()
salmon_model.predict(data)
end_time = time.time()
elapsed_time_pred_sal = (end_time - start_time) * 1000

## statsmodels predict
start_time = time.time()
statsmodels_results.predict(data)
end_time = time.time()
elapsed_time_pred_sm = (end_time - start_time) * 1000

## Salmon confidence intervals
start_time = time.time()
salmon_model.confidence_intervals(alpha=0.05)
end_time = time.time()
elapsed_time_ci_sal = (end_time - start_time) * 1000

## statsmodels confidence intervals
start_time = time.time()
statsmodels_results.conf_int(alpha=0.05, cols=None)
end_time = time.time()
elapsed_time_ci_sm = (end_time - start_time) * 1000

## salmon fit data2
start_time = time.time()
explanatory2 = C("Sex") + Q("Age") + Q("Exper") + Q("Senior") + Q("Bsal")
response2 = Q("Sal77")
salmon_model2 = LinearModel(explanatory2, response2)
salmon_model2.fit(data)
end_time = time.time()
elapsed_time_fit_sal2 = (end_time - start_time) * 1000

## statsmodels fit data2
start_time = time.time()
statsmodels_results2 = smf.ols("Sal77 ~ Sex + Age + Exper + Senior + Bsal", data=data).fit()
statsmodels_results2.summary()
end_time = time.time()
elapsed_time_fit_sm2 = (end_time - start_time) * 1000

## salmon predict2
start_time = time.time()
salmon_model2.predict(data)
end_time = time.time()
elapsed_time_pred_sal2 = (end_time - start_time) * 1000

## statsmodels predict2
start_time = time.time()
statsmodels_results2.predict(data)
end_time = time.time()
elapsed_time_pred_sm2 = (end_time - start_time) * 1000

## Salmon confidence intervals2
start_time = time.time()
salmon_model2.confidence_intervals(alpha=0.05)
end_time = time.time()
elapsed_time_ci_sal2 = (end_time - start_time) * 1000

## statsmodels confidence intervals2
start_time = time.time()
statsmodels_results2.conf_int(alpha=0.05, cols=None)
end_time = time.time()
elapsed_time_ci_sm2 = (end_time - start_time) * 1000

## salmon fit data3
start_time = time.time()
explanatory3 = C("Sex") + Q("Bsal") +  Q("Exper") + C("Sex") * Q("Bsal") + C("Sex") * Q("Exper")
response3 = Q("Sal77")
salmon_model3 = LinearModel(explanatory3, response3)
salmon_model3.fit(data)
end_time = time.time()
elapsed_time_fit_sal3 = (end_time - start_time) * 1000

## statsmodels fit data3
start_time = time.time()
statsmodels_results3 = smf.ols('Sal77 ~ Sex + Bsal + Exper + Sex*Bsal + Sex*Exper', data=data).fit()
statsmodels_results3.summary()
end_time = time.time()
elapsed_time_fit_sm3 = (end_time - start_time) * 1000

## salmon predict3
start_time = time.time()
salmon_model3.predict(data)
end_time = time.time()
elapsed_time_pred_sal3 = (end_time - start_time) * 1000

## statsmodels predict3
start_time = time.time()
statsmodels_results3.predict(data)
end_time = time.time()
elapsed_time_pred_sm3 = (end_time - start_time) * 1000

## Salmon confidence intervals3
start_time = time.time()
salmon_model3.confidence_intervals(alpha=0.05)
end_time = time.time()
elapsed_time_ci_sal3 = (end_time - start_time) * 1000

## statsmodels confidence intervals3
start_time = time.time()
statsmodels_results3.conf_int(alpha=0.05, cols=None)
end_time = time.time()
elapsed_time_ci_sm3 = (end_time - start_time) * 1000

# salmon fit data4
y = data2.loc[:,0]
X = data2.iloc[:,1:]
start_time = time.time()
salmon_model4 = LinearModel(X, y)
salmon_model4.fit()
end_time = time.time()
elapsed_time_fit_sal4 = (end_time - start_time) * 1000

## statsmodels fit data4
start_time = time.time()
statsmodels_results4 = sm.OLS(y, X).fit()
statsmodels_results4.summary()
end_time = time.time()
elapsed_time_fit_sm4 = (end_time - start_time) * 1000

# salmon predict4
start_time = time.time()
salmon_model4.predict(data2)
end_time = time.time()
elapsed_time_pred_sal4 = (end_time - start_time) * 1000

## statsmodels predict4
start_time = time.time()
statsmodels_results4.predict(X)
end_time = time.time()
elapsed_time_pred_sm4 = (end_time - start_time) * 1000

# Salmon confidence intervals4
start_time = time.time()
salmon_model4.confidence_intervals(alpha=0.05)
end_time = time.time()
elapsed_time_ci_sal4 = (end_time - start_time) * 1000

## statsmodels confidence intervals4
start_time = time.time()
statsmodels_results4.conf_int(alpha=0.05, cols=None)
end_time = time.time()
elapsed_time_ci_sm4 = (end_time - start_time) * 1000

In [ ]:
%%R

## R fit data
start_time = Sys.time()
R_model = lm("Sal77 ~ Bsal", data=data_r)
summary(R_model)
end_time = Sys.time()
elapsed_time_fit_r = (end_time - start_time) * 1000

## R predict
start_time = Sys.time()
predict(R_model)
end_time = Sys.time()
elapsed_time_pred_r = (end_time - start_time) * 1000

## R confidence intervals
start_time = Sys.time()
confint(R_model)
end_time = Sys.time()
elapsed_time_ci_r = (end_time - start_time) * 1000

# R fit data2
start_time = Sys.time()
R_model2 = lm("Sal77 ~ Sex + Age + Exper + Senior + Bsal", data=data_r)
summary(R_model2)
end_time = Sys.time()
elapsed_time_fit_r2 = (end_time - start_time) * 1000

## R predict2
start_time = Sys.time()
predict(R_model2)
end_time = Sys.time()
elapsed_time_pred_r2 = (end_time - start_time) * 1000

## R confidence intervals2
start_time = Sys.time()
confint(R_model2)
end_time = Sys.time()
elapsed_time_ci_r2 = (end_time - start_time) * 1000

# R fit data3
start_time = Sys.time()
R_model3 = lm("Sal77 ~ Sex + Bsal + Exper + Sex*Bsal + Sex*Exper", data=data_r)
summary(R_model3)
end_time = Sys.time()
elapsed_time_fit_r3 = (end_time - start_time) * 1000

## R predict3
start_time = Sys.time()
predict(R_model3)
end_time = Sys.time()
elapsed_time_pred_r3 = (end_time - start_time) * 1000

## R confidence intervals3
start_time = Sys.time()
confint(R_model3)
end_time = Sys.time()
elapsed_time_ci_r3 = (end_time - start_time) * 1000

# R fit data4
start_time = Sys.time()
R_model4 = lm("y ~ .", data=rdf)
summary(R_model)
end_time = Sys.time()
elapsed_time_fit_r4 = (end_time - start_time) * 1000

## R predict4
start_time = Sys.time()
predict(R_model4)
end_time = Sys.time()
elapsed_time_pred_r4 = (end_time - start_time) * 1000

## R confidence intervals4
start_time = Sys.time()
confint(R_model4)
end_time = Sys.time()
elapsed_time_ci_r4 = (end_time - start_time) * 1000

In [ ]:
elapsed_time_fit_r = %Rget elapsed_time_fit_r
elapsed_time_pred_r = %Rget elapsed_time_pred_r
elapsed_time_ci_r = %Rget elapsed_time_ci_r
elapsed_time_fit_r2 = %Rget elapsed_time_fit_r2
elapsed_time_pred_r2 = %Rget elapsed_time_pred_r2
elapsed_time_ci_r2 = %Rget elapsed_time_ci_r2
elapsed_time_fit_r3 = %Rget elapsed_time_fit_r3
elapsed_time_pred_r3 = %Rget elapsed_time_pred_r3
elapsed_time_ci_r3= %Rget elapsed_time_ci_r3
elapsed_time_fit_r4 = %Rget elapsed_time_fit_r4
elapsed_time_pred_r4 = %Rget elapsed_time_pred_r4
elapsed_time_ci_r4 = %Rget elapsed_time_ci_r4

table = [
["Fit",  elapsed_time_fit_sal, elapsed_time_fit_sm, elapsed_time_fit_r[0]],
["Predict", elapsed_time_pred_sal, elapsed_time_pred_sm, elapsed_time_pred_r[0]],
["CI", elapsed_time_ci_sal, elapsed_time_ci_sm, elapsed_time_ci_r[0]],
["Fit2",  elapsed_time_fit_sal2, elapsed_time_fit_sm2, elapsed_time_fit_r2[0]],
["Predict2", elapsed_time_pred_sal2, elapsed_time_pred_sm2, elapsed_time_pred_r2[0]],
["CI2", elapsed_time_ci_sal2, elapsed_time_ci_sm2, elapsed_time_ci_r2[0]],
["Fit3",  elapsed_time_fit_sal3, elapsed_time_fit_sm3, elapsed_time_fit_r3[0]],
["Predict3", elapsed_time_pred_sal3, elapsed_time_pred_sm3, elapsed_time_pred_r3[0]],
["CI3", elapsed_time_ci_sal3, elapsed_time_ci_sm3, elapsed_time_ci_r3[0]],
["Fit4",  elapsed_time_fit_sal4, elapsed_time_fit_sm4, elapsed_time_fit_r4[0]],
["Predict4", elapsed_time_pred_sal4, elapsed_time_pred_sm4, elapsed_time_pred_r4[0]],
["CI4", elapsed_time_ci_sal4, elapsed_time_ci_sm4, elapsed_time_ci_r4[0]],
]
print (tabulate(table, headers=["Test", "Salmon", "statsmodels", "R"]))

Test           Salmon    statsmodels            R
--------  -----------  -------------  -----------
Fit          3.02792       10.5846       2.36034
Predict      0.798702       2.62761      1.66464
CI           1.41668        0.926971     0.423193
Fit2         5.44786       22.6378       1.72496
Predict2     2.15983        7.47132      0.662565
CI2          1.49512        0.828505     0.325203
Fit3         8.28505       16.7458       1.41954
Predict3     3.59797        5.03325      0.555277
CI3          1.51277        1.02782      0.320435
Fit4      2218.52        7384.75      5115.14
Predict4   293.253         22.9208     185.411
CI4          2.16842        3.6304      45.7728
